Create CNN model and Optimize it using Keras Tuner

In [5]:
!pip install keras-tuner

# New section

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [10]:
print(tf.__version__)

2.6.0


In [11]:
fashion_mnist = keras.datasets.fashion_mnist

In [12]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [13]:
train_images = train_images/255.0
test_images = test_images/255.0

In [15]:
train_images[0].shape

(28, 28)

In [16]:
len(train_images)

60000

In [17]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [18]:
train_images.shape

(60000, 28, 28, 1)

In [46]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters = hp.Int("conv1_filter",min_value = 32 , max_value = 128 , step = 16),
                                kernel_size = hp.Choice("conv1_kernal",values = [3,5]),
                                activation = 'relu',
                                input_shape = (28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters = hp.Int("conv2_filter",min_value = 32 , max_value = 64,step = 16),
                                kernel_size = hp.Choice("conv2_kernal",values = [3,5]),
                                activation = "relu",
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int("dense_1_units",min_value = 32 , max_value = 128,step = 16),
                                activation="relu"
                            ),
                            keras.layers.Dense(10,activation = 'softmax')
  ])

  model.compile(optimizer = keras.optimizers.Adam(
      hp.Choice("learning_rate",values = [1e-2,1e-3])),
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy']
  )
  return model

In [21]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [50]:
tuner_search = RandomSearch(build_model,
                            objective="val_accuracy",
                            max_trials = 5,
                            directory='ouput',
                            project_name="Mnist Fashion"
                          )

INFO:tensorflow:Reloading Oracle from existing project ouput/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from ouput/Mnist Fashion/tuner0.json


In [48]:
tuner_search.search(train_images,train_labels,epochs = 3 , validation_split = 0.1)

INFO:tensorflow:Oracle triggered exit


In [49]:
tuner_search.results_summary()


Results summary
Results in ouput/Mnist Fashion
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv1_filter: 64
conv1_kernal: 5
conv2_filter: 64
conv2_kernal: 5
dense_1_units: 112
learning_rate: 0.1
Score: 0.1054999977350235
Trial summary
Hyperparameters:
conv1_filter: 112
conv1_kernal: 5
conv2_filter: 80
conv2_kernal: 5
dense_1_units: 112
learning_rate: 0.1
Score: 0.10499999672174454
Trial summary
Hyperparameters:
conv1_filter: 48
conv1_kernal: 3
conv2_filter: 112
conv2_kernal: 5
dense_1_units: 48
learning_rate: 0.1
Score: 0.10316666960716248
Trial summary
Hyperparameters:
conv1_filter: 64
conv1_kernal: 5
conv2_filter: 32
conv2_kernal: 5
dense_1_units: 32
learning_rate: 0.1
Score: 0.10266666859388351
Trial summary
Hyperparameters:
conv1_filter: 64
conv1_kernal: 3
conv2_filter: 96
conv2_kernal: 5
dense_1_units: 48
learning_rate: 0.1
Score: 0.10083333402872086


In [43]:
model = tuner_search.get_best_models(num_models=1)[0]

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        102464    
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2867312   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,972,570
Trainable params: 2,972,570
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.fit(train_images,train_labels,epochs = 10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 14s 8ms/step - loss: 2.3167 - accuracy: 0.0989 - val_loss: 2.3235 - val_accuracy: 0.1050
Epoch 5/10
1688/1688 [==============================] - 14s 8ms/step - loss: 2.3159 - accuracy: 0.0994 - val_loss: 2.3189 - val_accuracy: 0.1055
Epoch 6/10
1688/1688 [==============================] - 14s 8ms/step - loss: 2.3161 - accuracy: 0.0991 - val_loss: 2.3181 - val_accuracy: 0.0985
Epoch 7/10
1688/1688 [==============================] - 14s 8ms/step - loss: 2.3148 - accuracy: 0.1009 - val_loss: 2.3164 - val_accuracy: 0.0925
Epoch 8/10
1688/1688 [==============================] - 14s 8ms/step - loss: 2.3165 - accuracy: 0.0984 - val_loss: 2.3150 - val_accuracy: 0.1008
Epoch 9/10
1688/1688 [==============================] - 14s 8ms/step - loss: 2.3154 - accuracy: 0.1032 - val_loss: 2.3101 - val_accuracy: 0.1003
Epoch 10/10
1688/1688 [==============================] - 14s 8ms/step - loss: 2.3148 - accuracy: 0.1012 - val_loss: 2.3085 - val_a